In [1]:
import os
import numpy as np
from matplotlib.lines import Line2D
from scipy.stats import pearsonr, gaussian_kde, linregress, ttest_ind, sem, zscore
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import norm
from scipy.stats import percentileofscore
from sklearn.utils.validation import check_random_state
from math import factorial
from more_itertools import distinct_permutations
import statsmodels.api as sm

import matplotlib.pyplot as plt

import pandas as pd
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

import numpy as np
from scipy.stats import linregress
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold, ParameterGrid, train_test_split
from tqdm.notebook import tqdm

import warnings
import random
#from torch.utils.data import SubsetRandomSampler
from sklearn.utils import resample

from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score
from collections import Counter

from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score

warnings.filterwarnings("ignore")



In [2]:
from scipy.stats import chi2

def summarize_feature(df, feature, df_out, stat_cols=(10, 50), df_offset=2):

    d = df[df["Feature"] == feature]
    if d.empty:
        return df_out  

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            combined_p = 1 - chi2.cdf(d[col].mean(), df_chi)
            df_out.loc[feature, f"hl_pvalue_combined_{k}"] = combined_p

    # --- OR summary ---
    if "OR" in d.columns:
        or_mean = d["OR"].mean()
        or_std = d["OR"].std(ddof=1)
        df_out.loc[feature, "OR"] = or_mean
        df_out.loc[feature, "2.5%"] = or_mean - or_std
        df_out.loc[feature, "97.5%"] = or_mean + or_std

    # --- min/max de los IC originales (tal como lo estabas haciendo) ---
    if "2.5%" in d.columns:
        df_out.loc[feature, "minOR"] = d["2.5%"].min()
    if "97.5%" in d.columns:
        df_out.loc[feature, "maxOR"] = d["97.5%"].max()

    return df_out


features = ["Mono", "One", "Two", "Three", "Total"] 




def hosmer_lemeshow(y_true, y_prob, g):
    hl_df = pd.DataFrame({
        "observed": y_true,
        "predicted_probability": y_prob
    }).dropna()


    hl_df["group"] = pd.qcut(hl_df["predicted_probability"], g, duplicates="drop")

    hl_table = hl_df.groupby("group").apply(
        lambda x: pd.Series({
            "observed": x["observed"].sum(),
            "expected": x["predicted_probability"].sum(),
            "total": len(x)
        })
    )

    hl_table["observed_neg"] = hl_table["total"] - hl_table["observed"]
    hl_table["expected_neg"] = hl_table["total"] - hl_table["expected"]


    hl_statistic = (
        ((hl_table["observed"] - hl_table["expected"])**2) / hl_table["expected"] +
        ((hl_table["observed_neg"] - hl_table["expected_neg"])**2) / hl_table["expected_neg"]
    ).sum()


    dof = hl_table.shape[0] - 2
    p_value = 1 - chi2.cdf(hl_statistic, dof)

    return hl_statistic, p_value



def summarize_feature_by_covar(df, covar, feature, df_out, stat_cols=(10, 50), df_offset=2):
    d = df[(df["Covar"] == covar) & (df["Feature"] == feature)]
    if d.empty:
        return df_out

    idx = (covar, feature)

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            df_out.loc[idx, f"hl_pvalue_combined_{k}"] = 1 - chi2.cdf(d[col].mean(), df_chi)

    or_mean = d["OR"].mean()
    or_std  = d["OR"].std(ddof=1)
    df_out.loc[idx, "OR"] = or_mean
    df_out.loc[idx, "2.5%"] = or_mean - or_std
    df_out.loc[idx, "97.5%"] = or_mean + or_std

    df_out.loc[idx, "minOR"] = d["2.5%"].min()
    df_out.loc[idx, "maxOR"] = d["97.5%"].max()

    df_out.loc[idx, "n_iter"] = d.shape[0]

    return df_out


## Load data

In [3]:
data = pd.read_parquet('data/BBAG-cross.parquet')

In [4]:
Counter(data.country)

Counter({'Austria': 4320,
         'Belgium': 4942,
         'Czechia': 5575,
         'Switzerland': 2634,
         'Germany': 4567,
         'Denmark': 2609,
         'Estonia': 7522,
         'Spain': 4912,
         'France': 3640,
         'Hungary': 3010,
         'Italy': 4541,
         'Netherlands': 2546,
         'Poland': 3612,
         'Portugal': 1914,
         'Sweden': 2792,
         'Slovenia': 5291,
         'Luxembourg': 2104,
         'Greece': 2599,
         'Croatia': 2798,
         'Bulgaria': 1953,
         'Cyprus': 1205,
         'Finland': 1982,
         'Lithuania': 2033,
         'Latvia': 1684,
         'Malta': 1259,
         'Romania': 2083,
         'Slovakia': 2022})

In [5]:
vars_ = ['Mono', 'One',	'Two',	'Three', 'Total']


In [6]:
data.columns

Index(['record_id', 'country', 'year_acq_x', 'Sex_1F_2M', 'Education',
       'Barthel', 'Diabetes_1Y_0N', 'Hypertension_1Y_0N',
       'Heart_Disease_1Y_0N', 'Physical_activity_1Y_0N', 'Cognition',
       'Well_being_domain', 'Sleep_problems_1Y_0N', 'Audition_problems',
       'Vision_problems', 'Age', 'Mono', 'One', 'Two', 'Three', 'Total',
       'total_exposomes', 'sociopolitical', 'social_physical', 'GDP',
       'GDP_per_capita', 'gender_equal_l', 'Polution_conc_inv', 'Eq',
       'free_parties_l', 'inclu_suff_est', 'cred_elect_est', 'local_dem_est',
       'Migration', 'number_leng_inst', 'number_stable_inst', 'distance',
       'Proficiency', 'Educ_doct', 'Educ_upper_sec', 'Educ_master',
       'Educ_bach', 'GAP_corrected', 'GAP_bin', 'y_pred_corrected'],
      dtype='object')

## Odd ratios with macrosocials (new vars)

### Without Iteration

In [7]:
results_merge_df_all = data.copy()

covar_list = (
                'GDP_per_capita', 'GDP', 'Educ_doct', 'Educ_bach'
)

df_directions_odd_all = []
df_directions_cov_odd_all = []

for covar in covar_list:
    for i in vars_:

        c_results_merge_df_all = results_merge_df_all[[i, covar, 'GAP_bin']].dropna().reset_index(drop = True)
        
        y_ols = c_results_merge_df_all['GAP_bin']
        X_ols = c_results_merge_df_all[[i, covar]]

        X_train, X_test, y_train, y_test = train_test_split(
            X_ols, y_ols, test_size=0.2, random_state=42
        )

        scaler = MinMaxScaler((0.05, 0.95))

        X_train_scaled = pd.DataFrame(
            scaler.fit_transform(X_train),
            columns=X_train.columns,
            index=X_train.index
        )
        X_test_scaled = pd.DataFrame(
            scaler.transform(X_test),
            columns=X_test.columns,
            index=X_test.index
        )

        X_train_scaled['intercept'] = 1
        X_test_scaled['intercept'] = 1

        model = sm.Logit(y_train, X_train_scaled).fit(disp=0)

        params = model.params
        conf = np.exp(model.conf_int())
        conf['OR'] = np.exp(params)
        conf['z'] = model.tvalues
        conf['P>|z|'] = model.pvalues
        conf.columns = ['2.5%', '97.5%', 'OR', 'z', 'P>|z|']

        df_i = conf.loc[i:i].copy()
        df_i['Feature'] = i
        df_i['Covar'] = covar
        df_directions_odd_all.append(df_i)

        df_c = conf.loc[covar:covar].copy()
        df_c['Feature'] = f"{i}({covar})"
        df_c['Exposure'] = i          # opcional pero útil
        df_c['Covar'] = covar
        df_directions_cov_odd_all.append(df_c)

df_directions_odd = pd.concat(df_directions_odd_all, axis=0).reset_index(drop=True)
df_directions_cov_odd = pd.concat(df_directions_cov_odd_all, axis=0).reset_index(drop=True)

#df_directions_odd, df_directions_cov_odd


In [8]:
for covar in covar_list:
    c_dfOR = df_directions_odd[df_directions_odd.Covar == covar]
    c_dfOR.to_excel('Results/cross/cross_OR_-with-slovakia_new-covar-' + covar + '.xlsx')

    print(covar)
    print('Results/cross/cross_OR_-with-slovakia_new-covar-' + covar + '.xlsx')
    display(c_dfOR)

GDP_per_capita
Results/cross/cross_OR_-with-slovakia_new-covar-GDP_per_capita.xlsx


,2.5%,97.5%,OR,z,P>|z|,Feature,Covar
0,1.791987,2.049279,1.916320,19.003325,1.600750e-80,Mono,GDP_per_capita
1,0.591514,0.694690,0.641030,-10.841656,2.184802e-27,One,GDP_per_capita
2,0.510367,0.593270,0.550259,-15.557071,1.424714e-54,Two,GDP_per_capita
3,0.843387,1.002564,0.919538,-1.901901,5.718411e-02,Three,GDP_per_capita
4,0.488009,0.558152,0.521903,-18.980274,2.482993e-80,Total,GDP_per_capita


GDP
Results/cross/cross_OR_-with-slovakia_new-covar-GDP.xlsx


,2.5%,97.5%,OR,z,P>|z|,Feature,Covar
5,2.182891,2.476349,2.324995,26.220396,1.555784e-151,Mono,GDP
6,0.742215,0.870378,0.803746,-5.376344,7.601365e-08,One,GDP
7,0.433825,0.504123,0.467655,-19.837762,1.405782e-87,Two,GDP
8,0.552433,0.644908,0.596882,-13.069386,4.926124e-39,Three,GDP
9,0.403717,0.458035,0.430019,-26.206972,2.213088e-151,Total,GDP


Educ_doct
Results/cross/cross_OR_-with-slovakia_new-covar-Educ_doct.xlsx


,2.5%,97.5%,OR,z,P>|z|,Feature,Covar
10,1.870533,2.270587,2.060876,14.625381,1.934861e-48,Mono,Educ_doct
11,0.748378,0.916165,0.828032,-3.656683,2.554997e-04,One,Educ_doct
12,0.500896,0.610996,0.553213,-11.679575,1.621044e-31,Two,Educ_doct
13,0.623647,0.803265,0.707781,-5.352845,8.658193e-08,Three,Educ_doct
14,0.440220,0.534352,0.485008,-14.637343,1.622877e-48,Total,Educ_doct


Educ_bach
Results/cross/cross_OR_-with-slovakia_new-covar-Educ_bach.xlsx


,2.5%,97.5%,OR,z,P>|z|,Feature,Covar
15,1.879422,2.260557,2.061199,15.354887,3.284798e-53,Mono,Educ_bach
16,0.654446,0.790822,0.719410,-6.820034,9.101866e-12,One,Educ_bach
17,0.537241,0.674306,0.601885,-8.757886,1.989467e-18,Two,Educ_bach
18,0.712226,0.855662,0.780656,-5.290219,1.221698e-07,Three,Educ_bach
19,0.442580,0.532320,0.485381,-15.346861,3.717443e-53,Total,Educ_bach


### 1000-Iteration

In [9]:

results_merge_df_all = data.copy()

covar_list = (
                'GDP_per_capita', 'GDP', 'Educ_doct', 'Educ_bach'
)


n_boosts = 10  
test_size = 500 / results_merge_df_all.shape[0]


df_directions_odd_all = []
df_directions_cov_all = []


results_dict = {}  

for covar in covar_list:

    for boosts in range(n_boosts):

        for i in vars_:

            c_results_merge_df_all = results_merge_df_all[[i, covar, 'GAP_bin']].dropna().reset_index(drop = True)
            
            y_ols = c_results_merge_df_all['GAP_bin']
            X_ols = c_results_merge_df_all[[i, covar]]

            X_train, X_test, y_train, y_test = train_test_split(
                X_ols, y_ols,
                test_size=test_size,
                stratify=y_ols,
                random_state=boosts
            )

            scaler = MinMaxScaler((0.05, 0.95))

            X_train_scaled = pd.DataFrame(
                scaler.fit_transform(X_train),
                columns=X_train.columns,
                index=X_train.index
            )

            X_test_scaled = pd.DataFrame(
                scaler.transform(X_test),
                columns=X_test.columns,
                index=X_test.index
            )


            X_train_scaled["intercept"] = 1
            X_test_scaled["intercept"] = 1


            model = sm.Logit(y_train, X_train_scaled).fit(disp=0)
            y_test_pred = model.predict(X_test_scaled)

            key_y = (covar, i, "y")
            key_p = (covar, i, "ypred")
            if key_y not in results_dict:
                results_dict[key_y] = pd.Series(dtype=float)
                results_dict[key_p] = pd.Series(dtype=float)

            results_dict[key_y] = pd.concat([results_dict[key_y], y_test], axis=0)
            results_dict[key_p] = pd.concat([results_dict[key_p], y_test_pred], axis=0)

            params = model.params
            conf = np.exp(model.conf_int())
            conf["OR"] = np.exp(params)
            conf["z"] = model.tvalues
            conf["P>|z|"] = model.pvalues
            conf.columns = ["2.5%", "97.5%", "OR", "z", "P>|z|"]

            hl10_stat, hl10_p = hosmer_lemeshow(y_test, y_test_pred, g=10)
            hl50_stat, hl50_p = hosmer_lemeshow(y_test, y_test_pred, g=50)
            hl100_stat, hl100_p = hosmer_lemeshow(y_test, y_test_pred, g=100)

            df_i = conf.loc[i:i].copy()
            df_i["Feature"] = i
            df_i["Covar"] = covar
            df_i["boosts"] = boosts

            df_i["hl_statistic_10"] = hl10_stat
            df_i["hl_pvalue_10"] = hl10_p
            df_i["hl_statistic_50"] = hl50_stat
            df_i["hl_pvalue_50"] = hl50_p
            df_i["hl_statistic_100"] = hl100_stat
            df_i["hl_pvalue_100"] = hl100_p

            df_directions_odd_all.append(df_i)

            df_c = conf.loc[covar:covar].copy()
            df_c["Feature"] = i + "(" + covar + ")"
            df_c["Exposure"] = i
            df_c["Covar"] = covar
            df_c["boosts"] = boosts

            df_c["hl_statistic_10"] = hl10_stat
            df_c["hl_pvalue_10"] = hl10_p
            df_c["hl_statistic_50"] = hl50_stat
            df_c["hl_pvalue_50"] = hl50_p
            df_c["hl_statistic_100"] = hl100_stat
            df_c["hl_pvalue_100"] = hl100_p

            df_directions_cov_all.append(df_c)

df_directions_odd = pd.concat(df_directions_odd_all, axis=0).reset_index(drop=True)
df_directions_cov_odd = pd.concat(df_directions_cov_all, axis=0).reset_index(drop=True)



In [10]:


features = list(vars_) 
covars = df_directions_odd["Covar"].dropna().unique()

idx = pd.MultiIndex.from_product([covars, features], names=["Covar", "Feature"])
df_directions_odd_f = pd.DataFrame(index=idx)

for c in covars:
    for f in features:
        df_directions_odd_f = summarize_feature_by_covar(
            df_directions_odd, c, f, df_directions_odd_f,
            stat_cols=(10, 50, 100), 
            df_offset=2
        )

df_directions_odd_f = df_directions_odd_f.reset_index()
df_directions_odd_f;


In [11]:
for covar in covar_list:
    c_dfOR = df_directions_odd_f[df_directions_odd_f.Covar == covar]
    c_dfOR.to_excel('Results/cross/cross_OR_-with-slovakia_new-covar-' + covar + '-1000-iter.xlsx')

    print(covar)
    display(c_dfOR)

GDP_per_capita


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,OR,2.5%,97.5%,minOR,maxOR,n_iter
0,GDP_per_capita,Mono,0.120093,0.241568,0.978421,1.888720,1.884889,1.892551,1.772964,2.013408,10.0
1,GDP_per_capita,One,0.160783,0.245325,0.984038,0.625937,0.624702,0.627172,0.580331,0.675535,10.0
2,GDP_per_capita,Two,0.100510,0.317348,0.984775,0.569705,0.568751,0.570659,0.531375,0.610856,10.0
3,GDP_per_capita,Three,0.149327,0.287740,0.988907,0.932422,0.930200,0.934644,0.860004,1.010548,10.0
4,GDP_per_capita,Total,0.119961,0.246827,0.981715,0.529508,0.528433,0.530582,0.496684,0.564120,10.0


GDP


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,OR,2.5%,97.5%,minOR,maxOR,n_iter
5,GDP,Mono,0.431153,0.233480,0.976209,2.289576,2.284793,2.294359,2.158754,2.430881,10.0
6,GDP,One,0.085940,0.211844,0.963264,0.782243,0.780632,0.783853,0.725679,0.843956,10.0
7,GDP,Two,0.171095,0.143599,0.960584,0.485157,0.484416,0.485899,0.452723,0.520130,10.0
8,GDP,Three,0.079914,0.269018,0.969260,0.606812,0.605226,0.608398,0.563397,0.652301,10.0
9,GDP,Total,0.309712,0.264693,0.977497,0.436654,0.435742,0.437566,0.411250,0.463139,10.0


Educ_doct


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,OR,2.5%,97.5%,minOR,maxOR,n_iter
10,Educ_doct,Mono,0.245448,0.807041,0.999993,2.095028,2.089982,2.100074,1.913705,2.295678,10.0
11,Educ_doct,One,0.385815,0.786850,0.999990,0.847541,0.844405,0.850678,0.770062,0.933380,10.0
12,Educ_doct,Two,0.304580,0.871579,0.999987,0.541399,0.539613,0.543185,0.492417,0.594328,10.0
13,Educ_doct,Three,0.188045,0.784252,0.999996,0.682288,0.680321,0.684254,0.606077,0.767049,10.0
14,Educ_doct,Total,0.289694,0.820024,0.999991,0.477130,0.475980,0.478281,0.435430,0.522332,10.0


Educ_bach


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,OR,2.5%,97.5%,minOR,maxOR,n_iter
15,Educ_bach,Mono,0.194027,0.714534,0.999979,2.025228,2.015071,2.035384,1.855025,2.222823,10.0
16,Educ_bach,One,0.181804,0.649151,0.999958,0.739812,0.736921,0.742703,0.675100,0.808770,10.0
17,Educ_bach,Two,0.101694,0.698505,0.999962,0.625520,0.621978,0.629061,0.558531,0.699553,10.0
18,Educ_bach,Three,0.068667,0.635757,0.999954,0.762374,0.759329,0.765419,0.696615,0.833102,10.0
19,Educ_bach,Total,0.194727,0.694490,0.999972,0.493998,0.491536,0.496460,0.450082,0.539301,10.0


## Odd ratios with macrosocials (new vars-several covars)

### Without Iteration

In [12]:
results_merge_df_all = data.copy()

covar_name_group = ['globals',
                    'phisicals',
                    'sociopoliticals',
                    'economics',
                    'linguistics',
                    'educations']

covar_list = (['sociopolitical', 'social_physical'], 
              ['gender_equal_l', 'Polution_conc_inv', 'Eq'],
              ['free_parties_l',  'inclu_suff_est', 'cred_elect_est', 'local_dem_est'],
              ['GDP', 'GDP_per_capita'],
              ['number_leng_inst', 'number_stable_inst', 'distance', 'Migration'],
              ['Educ_doct', 'Educ_bach']
)

df_directions_odd_all = []

for covar_n in range(len(covar_list)):
    covar_name = covar_name_group[covar_n]
    covar = covar_list[covar_n]

    #print(covar, covar_name)
    
    for i in vars_:

        c_results_merge_df_all = results_merge_df_all[covar + [i, 'GAP_bin']].dropna().reset_index(drop = True)
        
        y_ols = c_results_merge_df_all['GAP_bin']
        X_ols = c_results_merge_df_all[[i] + covar]

        X_train, X_test, y_train, y_test = train_test_split(
            X_ols, y_ols, test_size=0.2, random_state=42
        )

        scaler = MinMaxScaler((0.05, 0.95))

        X_train_scaled = pd.DataFrame(
            scaler.fit_transform(X_train),
            columns=X_train.columns,
            index=X_train.index
        )
        X_test_scaled = pd.DataFrame(
            scaler.transform(X_test),
            columns=X_test.columns,
            index=X_test.index
        )

        X_train_scaled['intercept'] = 1
        X_test_scaled['intercept'] = 1

        model = sm.Logit(y_train, X_train_scaled).fit(disp=0)

        params = model.params
        conf = np.exp(model.conf_int())
        conf['OR'] = np.exp(params)
        conf['z'] = model.tvalues
        conf['P>|z|'] = model.pvalues
        conf.columns = ['2.5%', '97.5%', 'OR', 'z', 'P>|z|']

        df_i = conf.loc[i:i].copy()
        df_i['Feature'] = i
        df_i['Covar'] = covar_name
        df_directions_odd_all.append(df_i)


df_directions_odd = pd.concat(df_directions_odd_all, axis=0).reset_index(drop=True)

#df_directions_odd, df_directions_cov_odd


In [13]:
for covar_n in range(len(covar_list)):
    covar_name = covar_name_group[covar_n]
    covar = covar_list[covar_n]
    
    c_dfOR = df_directions_odd[df_directions_odd.Covar == covar_name]
    c_dfOR.to_excel('Results/cross/cross_OR_-with-slovakia_new-covar-several-covars-' + covar_name + '.xlsx')

    print(covar)
    display(c_dfOR)

['sociopolitical', 'social_physical']


,2.5%,97.5%,OR,z,P>|z|,Feature,Covar
0,1.947184,2.270312,2.102550,18.973753,2.811036e-80,Mono,globals
1,0.487958,0.578963,0.531516,-14.487300,1.457597e-47,One,globals
2,0.572908,0.696306,0.631600,-9.233861,2.610489e-20,Two,globals
3,0.796852,0.954894,0.872301,-2.959925,3.077139e-03,Three,globals
4,0.440352,0.513466,0.475506,-18.969898,3.024910e-80,Total,globals


['gender_equal_l', 'Polution_conc_inv', 'Eq']


,2.5%,97.5%,OR,z,P>|z|,Feature,Covar
5,2.434182,2.871005,2.643586,23.088016,6.109394e-118,Mono,phisicals
6,0.563385,0.666943,0.612981,-11.369521,5.931251e-30,One,phisicals
7,0.425736,0.517868,0.469547,-15.127087,1.073434e-51,Two,phisicals
8,0.702832,0.851317,0.773520,-5.252094,1.503793e-07,Three,phisicals
9,0.348264,0.410811,0.378247,-23.072764,8.692926e-118,Total,phisicals


['free_parties_l', 'inclu_suff_est', 'cred_elect_est', 'local_dem_est']


...

,2.5%,97.5%,OR,z,P>|z|,Feature,Covar
15,1.772331,2.041315,1.902074,17.836662,3.669731e-71,Mono,economics
16,0.507239,0.601558,0.552389,-13.641905,2.255885e-42,One,economics
17,0.523342,0.611300,0.565614,-14.378612,7.049431e-47,Two,economics
18,0.976542,1.190032,1.078015,1.489356,1.363936e-01,Three,economics
19,0.489928,0.564372,0.525834,-17.812139,5.689380e-71,Total,economics


['number_leng_inst', 'number_stable_inst', 'distance', 'Migration']


,2.5%,97.5%,OR,z,P>|z|,Feature,Covar
20,2.365685,2.778485,2.563790,22.945787,1.623354e-116,Mono,linguistics
21,0.754222,0.893476,0.820902,-4.565847,4.974808e-06,One,linguistics
22,0.454202,0.537370,0.494039,-16.438740,1.009972e-60,Two,linguistics
23,0.582080,0.701523,0.639016,-9.405201,5.193081e-21,Three,linguistics
24,0.360140,0.423017,0.390315,-22.917542,3.106272e-116,Total,linguistics


['Educ_doct', 'Educ_bach']


,2.5%,97.5%,OR,z,P>|z|,Feature,Covar
25,1.752637,2.163040,1.947055,12.414468,2.181314e-35,Mono,educations
26,0.677027,0.832892,0.750926,-5.419362,5.981225e-08,One,educations
27,0.532457,0.670850,0.597661,-8.733011,2.479814e-18,Two,educations
28,0.706800,0.920008,0.806388,-3.199626,1.376061e-03,Three,educations
29,0.462070,0.570239,0.513313,-12.427943,1.843172e-35,Total,educations


### 1000-Iteration

In [14]:
results_merge_df_all = data.copy()

covar_name_group = ['globals',
                    'phisicals',
                    'sociopoliticals',
                    'economics',
                    'linguistics',
                    'educations']

covar_list = (['sociopolitical', 'social_physical'], 
              ['gender_equal_l', 'Polution_conc_inv', 'Eq'],
              ['free_parties_l',  'inclu_suff_est', 'cred_elect_est', 'local_dem_est'],
              ['GDP', 'GDP_per_capita'],
              ['number_leng_inst', 'number_stable_inst', 'distance', 'Migration'],
              ['Educ_doct', 'Educ_bach']
)

n_boosts = 10  
test_size = 500 / results_merge_df_all.shape[0]

df_directions_odd_all = []

for covar_n in range(len(covar_list)):
    covar_name = covar_name_group[covar_n]
    covar = covar_list[covar_n]

    for boosts in range(n_boosts):

        for i in vars_:

            c_results_merge_df_all = results_merge_df_all[covar + [i, 'GAP_bin']].dropna().reset_index(drop = True)
            
            y_ols = c_results_merge_df_all['GAP_bin']
            X_ols = c_results_merge_df_all[[i] + covar]


            X_train, X_test, y_train, y_test = train_test_split(
                X_ols, y_ols,
                test_size=test_size,
                stratify=y_ols,
                random_state=boosts
            )

            scaler = MinMaxScaler((0.05, 0.95))

            X_train_scaled = pd.DataFrame(
                scaler.fit_transform(X_train),
                columns=X_train.columns,
                index=X_train.index
            )

            X_test_scaled = pd.DataFrame(
                scaler.transform(X_test),
                columns=X_test.columns,
                index=X_test.index
            )


            X_train_scaled["intercept"] = 1
            X_test_scaled["intercept"] = 1


            model = sm.Logit(y_train, X_train_scaled).fit(disp=0)
            y_test_pred = model.predict(X_test_scaled)

            key_y = (covar_name, i, "y")
            key_p = (covar_name, i, "ypred")
            if key_y not in results_dict:
                results_dict[key_y] = pd.Series(dtype=float)
                results_dict[key_p] = pd.Series(dtype=float)

            results_dict[key_y] = pd.concat([results_dict[key_y], y_test], axis=0)
            results_dict[key_p] = pd.concat([results_dict[key_p], y_test_pred], axis=0)

            params = model.params
            conf = np.exp(model.conf_int())
            conf["OR"] = np.exp(params)
            conf["z"] = model.tvalues
            conf["P>|z|"] = model.pvalues
            conf.columns = ["2.5%", "97.5%", "OR", "z", "P>|z|"]

            hl10_stat, hl10_p = hosmer_lemeshow(y_test, y_test_pred, g=10)
            hl50_stat, hl50_p = hosmer_lemeshow(y_test, y_test_pred, g=50)
            hl100_stat, hl100_p = hosmer_lemeshow(y_test, y_test_pred, g=100)

            df_i = conf.loc[i:i].copy()
            df_i["Feature"] = i
            df_i["Covar"] = covar_name
            df_i["boosts"] = boosts

            df_i["hl_statistic_10"] = hl10_stat
            df_i["hl_pvalue_10"] = hl10_p
            df_i["hl_statistic_50"] = hl50_stat
            df_i["hl_pvalue_50"] = hl50_p
            df_i["hl_statistic_100"] = hl100_stat
            df_i["hl_pvalue_100"] = hl100_p

            df_directions_odd_all.append(df_i)


df_directions_odd = pd.concat(df_directions_odd_all, axis=0).reset_index(drop=True)



In [15]:


features = list(vars_) 
covars = df_directions_odd["Covar"].dropna().unique()

idx = pd.MultiIndex.from_product([covars, features], names=["Covar", "Feature"])
df_directions_odd_f = pd.DataFrame(index=idx)

for c in covars:
    for f in features:
        df_directions_odd_f = summarize_feature_by_covar(
            df_directions_odd, c, f, df_directions_odd_f,
            stat_cols=(10, 50, 100), 
            df_offset=2
        )

df_directions_odd_f = df_directions_odd_f.reset_index()
df_directions_odd_f;


In [16]:
for covar_n in range(len(covar_list)):
    covar_name = covar_name_group[covar_n]
    covar = covar_list[covar_n]
    
    c_dfOR = df_directions_odd_f[df_directions_odd_f.Covar == covar_name]
    c_dfOR.to_excel('Results/cross/cross_OR_-with-slovakia_new-covar-several-covars-' + covar_name + '-1000-iter.xlsx')

    print(covar)
    display(c_dfOR)

['sociopolitical', 'social_physical']


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,OR,2.5%,97.5%,minOR,maxOR,n_iter
0,globals,Mono,0.171243,0.202137,0.974906,2.103428,2.097185,2.109670,1.957426,2.264909,10.0
1,globals,One,0.083259,0.279294,0.977208,0.522365,0.521260,0.523471,0.482579,0.566355,10.0
2,globals,Two,0.186580,0.264433,0.980285,0.652970,0.651033,0.654907,0.596355,0.716765,10.0
3,globals,Three,0.393554,0.223752,0.965506,0.868553,0.865632,0.871475,0.795393,0.945751,10.0
4,globals,Total,0.171209,0.209385,0.976073,0.475264,0.473855,0.476672,0.441357,0.510727,10.0


['gender_equal_l', 'Polution_conc_inv', 'Eq']


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,OR,2.5%,97.5%,minOR,maxOR,n_iter
5,phisicals,Mono,0.061335,0.254414,0.976845,2.603903,2.595602,2.612205,2.411101,2.819051,10.0
6,phisicals,One,0.196840,0.343321,0.987533,0.602981,0.601733,0.604229,0.557406,0.652970,10.0
7,phisicals,Two,0.223220,0.342846,0.978782,0.491096,0.489717,0.492475,0.447959,0.538355,10.0
8,phisicals,Three,0.124459,0.218772,0.975900,0.779820,0.777044,0.782595,0.710187,0.853936,10.0
9,phisicals,Total,0.055951,0.285891,0.975067,0.383988,0.382766,0.385211,0.354657,0.414717,10.0


['free_parties_l', 'inclu_suff_est', 'cred_elect_est', 'local_dem_est']


...

,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,OR,2.5%,97.5%,minOR,maxOR,n_iter
15,economics,Mono,0.121670,0.293345,0.985899,1.859689,1.856049,1.863328,1.740022,1.989898,10.0
16,economics,One,0.176913,0.498003,0.991378,0.533123,0.531666,0.534580,0.492189,0.579198,10.0
17,economics,Two,0.222813,0.322497,0.985550,0.590178,0.589225,0.591131,0.549110,0.634258,10.0
18,economics,Three,0.059227,0.274770,0.983699,1.114419,1.110827,1.118011,1.014793,1.222249,10.0
19,economics,Total,0.107275,0.291610,0.986026,0.537803,0.536750,0.538855,0.502574,0.574837,10.0


['number_leng_inst', 'number_stable_inst', 'distance', 'Migration']


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,OR,2.5%,97.5%,minOR,maxOR,n_iter
20,linguistics,Mono,0.153557,0.460417,0.981381,2.504000,2.498006,2.509995,2.322337,2.700759,10.0
21,linguistics,One,0.211464,0.205852,0.967383,0.798928,0.797222,0.800634,0.737945,0.865788,10.0
22,linguistics,Two,0.064365,0.198879,0.976646,0.516289,0.515424,0.517155,0.477567,0.558215,10.0
23,linguistics,Three,0.049459,0.175421,0.973909,0.652489,0.650602,0.654375,0.597374,0.712372,10.0
24,linguistics,Total,0.152838,0.424364,0.985217,0.399595,0.398637,0.400552,0.370464,0.430861,10.0


['Educ_doct', 'Educ_bach']


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,OR,2.5%,97.5%,minOR,maxOR,n_iter
25,educations,Mono,0.232961,0.795618,0.999992,2.007648,2.001525,2.013772,1.817698,2.216683,10.0
26,educations,One,0.399285,0.800730,0.999989,0.773490,0.770633,0.776348,0.701065,0.853358,10.0
27,educations,Two,0.238420,0.813605,0.999993,0.570533,0.568339,0.572727,0.511033,0.636674,10.0
28,educations,Three,0.126321,0.710950,0.999990,0.767689,0.765162,0.770217,0.678553,0.867816,10.0
29,educations,Total,0.242874,0.794429,0.999993,0.497864,0.496342,0.499387,0.450920,0.549880,10.0
